In [1]:
import torch
from torch import nn
from torch.nn import NLLLoss
from torch.optim import Adam

/home2/abhisingam/miniconda3/envs/inlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class AMORE (nn.Module):
    def __init__(self):
        self.word_input_size = 300
        self.speaker_input_size = 65
        self.speaker_embedding_size = 134
        self.lstm_hidden_size = 459        
        
        self.SpeakerEmbed = nn.Embedding(self.speaker_input_size, self.speaker_embedding_size)
        self.LSTM = nn.LSTM(input_size = self.word_input_size + self.speaker_embedding_size, hidden_size = self.lstm_hidden_size, bidirectional=True)
        self.Classifier = nn.Linear(self.lstm_hidden_size, self.speaker_embedding_size)
        
        
    def forward(self, x):
        x = np.array(x)
        speak_embed = self.SpeakerEmbed(x[:,1])